In [18]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
path = 'clean_weather.csv'

weather_df = pd.read_csv(path)

del weather_df['Unnamed: 0']

weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mujiayingzi,42.12,118.78,7.66,84,0,7.87,CN,1608091722
1,Ponta do Sol,32.67,-17.10,60.28,67,20,1.12,PT,1608091722
2,Camacha,33.08,-16.33,64.40,77,75,17.22,PT,1608091722
3,Kahului,20.89,-156.47,73.56,61,40,18.34,US,1608091722
4,Avarua,-21.21,-159.78,82.40,78,100,11.41,CK,1608091722


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Lat", "Lng"]].astype(float)

humidity = weather_df["Humidity"]


In [12]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
#ideal_weather_df = weather_df.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) & (weather_df["Max Temp"] >= 70) & (weather_df["Max Temp"] <= 80)].dropna()

ideal_weather_df = weather_df.loc[weather_df["Wind Speed"] <= 10]
                                   
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Max Temp"] >= 70]

ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Max Temp"] <= 80]

ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Cloudiness"] == 0]

ideal_weather_df = ideal_weather_df.dropna(how='any')

ideal_weather_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
84,Busselton,-33.65,115.33,75.99,40,0,5.99,AU,1608091735
98,Myitkyina,25.38,97.40,73.99,42,0,2.17,MM,1608091737
146,San Lorenzo,-25.33,-57.53,78.80,83,0,6.17,PY,1608091745
288,Tura,25.52,90.22,72.97,55,0,2.35,IN,1608091769
321,Salalah,17.02,54.09,73.40,46,0,9.17,OM,1608091774
546,Mandera,3.94,41.87,75.24,38,0,7.70,KE,1608091814


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
#Create the hotel_df variable
hotel_df = ideal_weather_df
hotel_df.insert(0,'Hotel Name','')
hotel_df

,Hotel Name,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
84,,Busselton,-33.65,115.33,75.99,40,0,5.99,AU,1608091735
98,,Myitkyina,25.38,97.40,73.99,42,0,2.17,MM,1608091737
146,,San Lorenzo,-25.33,-57.53,78.80,83,0,6.17,PY,1608091745
288,,Tura,25.52,90.22,72.97,55,0,2.35,IN,1608091769
321,,Salalah,17.02,54.09,73.40,46,0,9.17,OM,1608091774
546,,Mandera,3.94,41.87,75.24,38,0,7.70,KE,1608091814


In [29]:
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key}

ideal_cities = len(ideal_weather_df)
  
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

hotels =[]

for x,row in ideal_weather_df.iterrows():
    lat = row['Lat']
    lon = row['Lng']
    params['location'] = f'{lat},{lon}'
    response= requests.get(base_url, params=params ).json()
    hotels.append(response['results'][0]['name'])
    

In [31]:
ideal_weather_df['Hotel Name'] = hotels
ideal_weather_df

,Hotel Name,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
84,Esplanade Hotel,Busselton,-33.65,115.33,75.99,40,0,5.99,AU,1608091735
98,Cartel Hotel,Myitkyina,25.38,97.40,73.99,42,0,2.17,MM,1608091737
146,Factoria Hotel,San Lorenzo,-25.33,-57.53,78.80,83,0,6.17,PY,1608091745
288,Hotel Polo Orchid,Tura,25.52,90.22,72.97,55,0,2.35,IN,1608091769
321,InterCityHotel Salalah,Salalah,17.02,54.09,73.40,46,0,9.17,OM,1608091774
546,Granada hotel Mandera,Mandera,3.94,41.87,75.24,38,0,7.70,KE,1608091814


In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather_df.iterrows()]
locations = ideal_weather_df[["Lat", "Lng"]]

In [35]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=[f'{text}'for text in hotel_info])
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))